<a href="https://colab.research.google.com/github/yiangMao/CODE-BREAKER-KARAOKE/blob/main/CODE_BREAKER_KARAOKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import itertools
import time

# generate guesses for computer
def computer_guess_sequence(checked_number):
    digits = '123456789'
    guesses = []

    # after 3 numbers varified, generate all perms of the 3 numbers
    if(len(checked_number) == 3):
      guesses = []
      for perm in itertools.permutations(checked_number, 3):
        guess = ''.join(perm)
        guesses.append(guess)

    # generate 111,222,333,...999
    for d in digits:
        guess = d * 3
        guesses.append(guess)

    return guesses

# Generate feedback

# F1: hard version
def get_feedback(secret, guess):
    feedback = ''
    # +/- counter
    plus = 0
    minus = 0
    secret_list = str(secret)
    guess_list = str(guess)

    if secret == guess:
       return '+++'
    # loop through each index
    for i in range(3):
      if guess[i] in secret:
        if secret[i] == guess[i]:
          plus+=1
        else:
          minus+=1

    feedback = '+' * plus + '-' * minus
    return feedback

# F2: easy version
# def get_feedback(secret, guess):

#     secret = str(secret)
#     guess = str(guess)
#     feedback = ''

#     if secret == guess:
#         return '+++'

#     # loop through each index
#     for i in range(3):
#       if guess[i] in secret:
#         if secret[i] == guess[i]:
#           feedback += '+'
#         else:
#           feedback += '-'

#     return feedback


# Code breaker Karaoke

class CodeBreakerKaraoke:
    def __init__(self):
        self.players = []
        self.player_guess_counts = {}
        self.player_guess_times = {}
        self.player_guess_counts_total = {}
        self.player_guess_times_total = {}
        self.player_win_counts = {}
        self.player_secret = ''
        #self.player_win_number = 0
        #self.player_win = False
        self.computer_guess_counts = {}
        self.current_player_index = 0
        self.current_guess_index = 0
        self.computer_guess_sequence = [] # This will be filtered based on feedback
        self.computer_secret = ''
        self.start_time = 0
        self.possible_computer_guesses = [] # Store potential guesses for computer
        self.computer_verified_number = []
        self.computer_win_count = 0
        self.max_round = 0
        self.current_round = 0


    # register players in game, set number of players, player names
    # set number of players
    def register_players(self):
        while True:
            try:
                print("Welcome to Code Breaker Karaoke!")
                num_players_str = input("Enter number of players (2-4): ")
                num_players = int(num_players_str)
                if 2 <= num_players <= 4:
                    break
                else:
                    print("Please enter a number between 2 and 4.")
            except ValueError:
                print("Invalid input. Please enter a number.")

        self.max_round = num_players * 2
        self.players = []
        self.player_guess_counts = {}
        self.player_guess_times = {}
        self.computer_guess_counts = {}
        self.player_win_counts = {}
        self.player_guess_counts_total = {}
        self.player_guess_times_total = {}

        # initialize players, set player names
        for i in range(num_players):
            while True:
                name = input(f"Enter name for Player {i+1}: ")
                if name:
                    self.players.append(name)
                    self.player_guess_counts[name] = 0
                    self.player_guess_times[name] = 0.0
                    self.computer_guess_counts[name] = 0
                    self.player_win_counts[name] = 0
                    self.player_guess_counts_total[name] = 0
                    self.player_guess_times_total[name] = 0.0
                    break
                else:
                    print("Player name cannot be empty.")

        # display player names
        print("\nPlayers registered:", ", ".join(self.players))
        self.play_round()

    def play_round(self):
        # checking if all players finished their round
        if self.current_player_index >= len(self.players):
          self.current_round += 1
          self.show_leaderboard()
          if self.current_round > self.max_round:
            self.show_final_leaderboard()
            return
          elif max(self.player_win_counts.values()) == 3:
            self.show_final_leaderboard()
            return
          else:
            next_round = input("Next round? (yes/no): ").lower()
            while next_round not in ['yes', 'no']:
              print("Invalid input. Please enter 'yes' or 'no'.")
              next_round = input("Next round? (yes/no): ").lower()
            if next_round == 'yes':
              self.current_player_index = 0
              for player in self.players:
                self.player_guess_counts[player] = 0
                self.player_guess_times[player] = 0.0
                self.computer_guess_counts[player] = 0
            else:
              self.show_final_leaderboard()
              return

        # getting to current player
        player = self.players[self.current_player_index]
        print(f"\n--- {player}'s Round ---")

        # Computer's turn to guess the player's secret code
        self.player_secret = input(f"{player}, enter your 3-digit secret code (unique digits 1-9): ")

        # checking for length = 3, is digit, no repeat, no 0s
        while len(self.player_secret) != 3 or \
        not self.player_secret.isdigit() or \
        len(set(self.player_secret)) != 3 or \
        '0' in self.player_secret:
            self.player_secret = input("Invalid code. Enter a 3-digit secret code (unique digits 1-9): ")

        print(f"\nComputer is now guessing {player}'s code.")

        # initialize guess count for current player
        self.computer_guess_counts[player] = 0
        self.computer_guess_index = 0 # Reset index for filtering

        self.run_computer_turn()

    # computer's turn
    def run_computer_turn(self):
        player = self.players[self.current_player_index]
        all_number_verifiied = False

        # generate initial guesses, 111,222,333,...999
        self.possible_computer_guesses = computer_guess_sequence([])

        # loop through possible guesses
        while self.possible_computer_guesses:
            current_guess = self.possible_computer_guesses[0] # Get the first possible guess
            self.computer_guess_counts[player] += 1 # increasment computer guess count
            print(f"Computer's Guess #{self.computer_guess_counts[player]}: {current_guess}")

            # getting feedback from player
            feedback = input("Enter feedback (+ for correct pos, - for correct digit wrong pos, empty for wrong): ")
            while feedback != "" and (len(feedback) != 3 or any(char not in '+- ' for char in feedback)):
                feedback = input("Invalid feedback. Enter + for correct pos, - for correct digit wrong pos, empty for wrong: ")

            # currect guess
            if feedback == '+++':
                print(f"Computer guessed {player}'s code in {self.computer_guess_counts[player]} guesses.")
                self.generate_computer_secret() # Move to player's turn
                return

            elif feedback == '':
                # remove current guess from possible guess list
                self.possible_computer_guesses.pop(0)
            elif '+' in feedback:
                # remove current guess from self.possible_computer_guesses
                self.possible_computer_guesses.pop(0)
                # add current number in guess to verified numbers
                self.computer_verified_number.append(current_guess[0])
            else:
                # remove current guess from self.possible_computer_guesses
                self.possible_computer_guesses.pop(0)

            # case when all 3 numbers found for the first time
            if len(self.computer_verified_number) == 3 and not all_number_verifiied:
                # replace possible_computer_guesses with perm of 3 correct numbers
                self.possible_computer_guesses = computer_guess_sequence(self.computer_verified_number)
                self.computer_verified_number = []
                all_number_verified = True

        # If the loop finishes without the computer guessing the code, case when player input wrong feedback
        print(f"Computer could not guess {player}'s code.")
        self.generate_computer_secret() # Move to player's turn

    # generate computer secret number for player
    def generate_computer_secret(self):
        player = self.players[self.current_player_index]
        print(f"\n{player}, it's your turn to guess the computer's secret code.")
        self.computer_secret = ''.join(random.sample('123456789', 3)) # Computer generates a new secret code
        print("Computer has generated a new 3-digit secret code.(",self.computer_secret,")")

        # initilize rount player guess count and time
        self.player_guess_counts[player] = 0
        self.start_time = time.time()
        # start player turn
        self.run_player_turn()

    # player turn
    def run_player_turn(self):
        player = self.players[self.current_player_index]
        while True:
            guess = input("Enter your guess (3 unique digits from 1 to 9): ")
            if len(guess) != 3 or not guess.isdigit() or len(set(guess)) != 3 or '0' in guess:
                print("Invalid guess. Use 3 unique digits from 1 to 9.")
                continue
            # increament rount player guess, and total player guess
            self.player_guess_counts[player] += 1
            self.player_guess_counts_total[player] += 1

            # get feedback from current guess
            feedback = get_feedback(self.computer_secret, guess)

            # correct guess
            if feedback == '+++':
                guessTime = time.time() - self.start_time
                self.player_guess_times[player] = guessTime

                # increament total player guess time
                self.player_guess_times_total[player] += guessTime

                print(f"Correct! {player} guessed the computer's code in {self.player_guess_counts[player]} guesses and {guessTime:.2f} seconds.")
                if self.player_guess_counts[player] < self.computer_guess_counts[player]:
                    self.player_win_counts[player] += 1
                # else:
                #     self.computer_win_count += 1

                self.next_player()
                break # Exit player's turn loop
            # wrong guess
            else:
                print(f"Feedback: {feedback}")
                print(f"You have made {self.player_guess_counts[player]} guesses.")
                if self.player_guess_counts[player] == self.computer_guess_counts[player]:
                  guessTime = time.time() - self.start_time
                  self.player_guess_times[player] = guessTime

                  # increament total player guess time
                  self.player_guess_times_total[player] += guessTime
                  print("The computer wins this round, better luck next time!")
                  self.next_player()
                  break # Exit player's turn loop

    # get to next player, increament player index
    def next_player(self):
        self.current_player_index += 1
        self.play_round() # Start the next player's round

    # display leaderboard
    def show_leaderboard(self):
        print("\n--- Current Round ---")
        for player in self.players:
            p_guesses = self.player_guess_counts.get(player, 0)
            p_time = self.player_guess_times.get(player, 0.0)
            c_guesses = self.computer_guess_counts.get(player, 0)
            print(f"{player} | Player Guesses: {p_guesses} | Time: {p_time:.2f} sec | Computer Guesses: {c_guesses}")
        print("---------------------------")
        print("\n--- Current Scoreboard ---")
        for player in self.players:
          p_win = self.player_win_counts.get(player, 0)
          p_total_time = self.player_guess_times_total.get(player, 0.0)
          p_total_guesses = self.player_guess_counts_total.get(player, 0)
          print(f"{player} | Score: {p_win} | Total Time: {p_total_time:.2f} sec | Total Guess: {p_total_guesses}")


    def show_final_leaderboard(self):
        print("\n--- Final Leaderboard ---")
        # sort players based on wins, time, guesses
        sorted_players = sorted(self.players, key=lambda p: (self.player_win_counts.get(p,float('inf')),self.player_guess_times.get(p, float('inf')), self.player_guess_counts.get(p, float('inf'))))

        # get winner
        winner = sorted_players[0]

        # display each player in order
        for player in sorted_players:
            p_win = self.player_win_counts.get(player, 0)
            p_total_time = self.player_guess_times_total.get(player, 0.0)
            p_total_guesses = self.player_guess_counts_total.get(player, 0)
            print(f"{player} | Score: {p_win} | Total Time: {p_total_time:.2f} sec  | Total Guess: {p_total_guesses}")
        print("-------------------------")

        # display winner
        print(f"\nWinner: {winner}")

        while True:
            play_again = input("Play again? (yes/no): ").lower()
            if play_again == 'yes':
                self.__init__() # Reset game state
                self.register_players() # Start a new game
                break
            elif play_again == 'no':
                print("\nThanks for playing!")
                break
            else:
                print("Invalid input. Please enter 'yes' or 'no'.")


# Main execution block for text-based game
if __name__ == "__main__":
    game = CodeBreakerKaraoke()
    game.register_players()

Welcome to Code Breaker Karaoke!
Enter number of players (2-4): 2
Enter name for Player 1: p1
Enter name for Player 2: p2

Players registered: p1, p2

--- p1's Round ---
p1, enter your 3-digit secret code (unique digits 1-9): 123

Computer is now guessing p1's code.
Computer's Guess #1: 111
Enter feedback (+ for correct pos, - for correct digit wrong pos, empty for wrong): +--
Computer's Guess #2: 222
Enter feedback (+ for correct pos, - for correct digit wrong pos, empty for wrong): +--
Computer's Guess #3: 333
Enter feedback (+ for correct pos, - for correct digit wrong pos, empty for wrong): +--
Computer's Guess #4: 123
Enter feedback (+ for correct pos, - for correct digit wrong pos, empty for wrong): +++
Computer guessed p1's code in 4 guesses.

p1, it's your turn to guess the computer's secret code.
Computer has generated a new 3-digit secret code.( 294 )
Enter your guess (3 unique digits from 1 to 9): 123
Feedback: -
You have made 1 guesses.
Enter your guess (3 unique digits fro